In [1]:
import time
import copy
import gc
import numpy as np
import argparse
from pyspark.sql.functions import countDistinct, col, count, isnan
from datetime import date, datetime, timedelta
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql import functions as F
from pyspark.sql.functions import sum, max, avg, countDistinct, col, lit, when
from pyspark.sql.types import StringType, IntegerType, StructType, StructField, DoubleType, DateType, TimestampType, ArrayType
from pyspark.sql import Window
from functools import reduce
from itertools import chain
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml.functions import vector_to_array
import numpy as np
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType
# initiate the spark session
spark = SparkSession \
    .builder \
    .config("testKey", "testValue") \
    .enableHiveSupport() \
    .getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/29 11:11:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df = spark.read.parquet("/Users/chongbei/Downloads/mobr-data/mobr0127.parquet")


In [3]:
df.printSchema()

root
 |-- marketplace_id: integer (nullable = true)
 |-- campaign_id: long (nullable = true)
 |-- metric_dates: array (nullable = true)
 |    |-- element: date (containsNull = true)
 |-- estimated_spend_distr_sorted_by_date: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- estimated_full_day_spend_mean: double (nullable = true)
 |-- realized_spend_sorted_by_date: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- mo_impressions: long (nullable = true)
 |-- mo_clicks: long (nullable = true)
 |-- mo_sales: double (nullable = true)
 |-- mo_spend: double (nullable = true)
 |-- sum_realized_spend: double (nullable = true)
 |-- sum_realized_ops_1d: double (nullable = true)
 |-- in_budget_seconds_mean: double (nullable = true)
 |-- active_campaign_seconds_mean: double (nullable = true)
 |-- roas: double (nullable = true)
 |-- advertiser_id: long (nullable = true)
 |-- daily_budget_usd: double (nullable = true)
 |-- max_effective_budget_sor

In [6]:
df.show(truncate=False, vertical=True)

-RECORD 0--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 marketplace_id                       | 1                                                                                                                                                                                                                              
 campaign_id                          | 200029915781498                                                                                                                                                                                                                
 metric_dates                         | [2024-01-18, 2024-01-19, 2024-01-20, 2024-01-21, 2024-01-22, 2024-01-23, 2024-01-24]                                                                                    